# ПРОЕКТ: Нейросеть для автодополнения текстов
* Реализована и обучена модель на основе рекуррентных нейронных сетей
* Взята более «тяжёлая» предобученную модель из Transformers
* Проведена оценка эфктивности двух моделей

In [3]:
import torch
from transformers import pipeline
from transformers import GPT2Tokenizer
from torch.utils.data import DataLoader

from src.data_utils import *
from src.next_token_dataset import NextTokenDataset
from src.lstm_model import LSTMTextGenerator
from src.lstm_train import train_lstm_model 
from src.lstm_eval import evaluate_lstm
from src.transformer_eval_pipline import evaluate_transformer

# Подготовка данных

In [2]:
# Подготовка данных
save_tweets_to_csv()
process_dataset()
split_dataset()

Данные успешно переформатированы в data/raw_dataset.csv
Очищенный датасет сохранен: data/dataset_processed.csv

Датасет размерностью 1596876 строк разделен на:
train: 1277500 
val: 159688 
test: 159688


In [2]:
# DataLoader

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.save_pretrained("models/tokenizer/")

train_texts = pd.read_csv("data/train.csv")['tweet'].tolist()[:638750]
val_texts = pd.read_csv("data/val.csv")['tweet'].tolist()[:79844]
test_texts = pd.read_csv("data/test.csv")['tweet'].tolist()[:79844]

train_dataset = NextTokenDataset(train_texts, tokenizer, max_length=20)
val_dataset = NextTokenDataset(val_texts, tokenizer, max_length=20)
test_dataset = NextTokenDataset(test_texts, tokenizer, max_length=20)

def collate_fn(batch, pad_token_id=50256):
    import torch
    from torch import nn

    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    labels = [torch.tensor(item['labels']) for item in batch]

    # Паддинг до максимальной длины в батче
    input_ids = nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=pad_token_id)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=pad_token_id)

    return input_ids, labels

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, collate_fn=collate_fn)

# Реализация рекуррентной сети

In [8]:
# Обучение LSTM

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = train_lstm_model(train_loader, val_loader, vocab_size=tokenizer.vocab_size, device=device)

Epoch 1: 100%|██████████| 2486/2486 [32:59<00:00,  1.26it/s]


Epoch 1, Loss: 6.6611


Epoch 2: 100%|██████████| 2486/2486 [44:54<00:00,  1.08s/it]


Epoch 2, Loss: 5.9016


Epoch 3: 100%|██████████| 2486/2486 [33:11<00:00,  1.25it/s]

Epoch 3, Loss: 5.6904
Модель сохранена: models/lstm_model.pth


# Тренировка рекуррентной сети

In [3]:
# Устройство (CPU или GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Инициализация модели
model = LSTMTextGenerator(vocab_size=tokenizer.vocab_size).to(device)
model.load_state_dict(torch.load('models/lstm_model.pth', map_location=device))
model.to(device)  # Переместите модель на нужное устройство
model.eval()  # Переключите модель в режим оценки

lstm_rouge1, lstm_rouge2 = evaluate_lstm(model, val_loader, tokenizer, device=device)

C:\Users\vasal\AppData\Local\Temp\ipykernel_1424\802171707.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/lstm_model.pth', map_

LSTM ROUGE-1: 0.3534
LSTM ROUGE-2: 0.3050


Значения LSTM ROUGE-1 = 0.3534 и LSTM ROUGE-2 = 0.3050 указывает на то, что качество сгенерированного текста относительно низкое. 
<br>Модель может не передавать ключевые идеи или факты. 

# Использование предобученного трансформенра

In [3]:
# Устройство (CPU или GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transformer_rouge1, transformer_rouge2 = evaluate_transformer(val_loader, tokenizer, device=device, max_examples=1000)

DistilGPT-2: 100%|██████████| 1000/1000 [02:20<00:00,  7.11it/s]

DistilGPT-2 (на 1000 примерах):
  ROUGE-1: 0.6672
  ROUGE-2: 0.6164


Значения ROUGE-1 и ROUGE-2 указывают на то, что модель генерирует текст, который достаточно близок к эталонному. 
<br>Также модель хорошо справляется с задачей извлечения ключевой информации и соблюдения структуры предложений.

# Сравнение работы двух моделей

In [4]:
# Сравнение результатов LSTM и DistilGPT-2
# Загружаем модель DistilGPT-2

# Инициализация модели
model = LSTMTextGenerator(vocab_size=tokenizer.vocab_size).to(device)
model.load_state_dict(torch.load('models/lstm_model.pth', map_location=device))
model.to(device)  # Переместите модель на нужное устройство

generator_DistilGPT = pipeline("text-generation", model="distilgpt2", tokenizer=tokenizer)

# Примеры промптов — начала фраз
examples = [
    "My friend and I play",
    "I know who",
    "I want to",
    "We have a flat in",
    "I cook dinner"
]
print("Оценка автодополнения LSTM:")
for prompt in examples:
    generated = model.generate(tokenizer, prompt, max_length=20, device=device)
    print(f"Промпт: {prompt}")
    print(f"Дополнение LSTM: {generated}")
print("\nОценка автодополнения DistilGPT:")
for prompt in examples:
    result = generator_DistilGPT(prompt, max_length=20, do_sample=True, top_k=50)
    generated = result[0]['generated_text']
    print(f"Промпт: {prompt}")
    print(f"Дополнение DistilGPT: {generated}")

C:\Users\vasal\AppData\Local\Temp\ipykernel_6704\989992326.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/lstm_model.pth', map_

Оценка автодополнения LSTM:
Промпт: My friend and I play
Дополнение LSTM: my friend and i play the new moon trailer i m so tired i m so tired i m so
Промпт: I know who
Дополнение LSTM: i know who i m so tired i m so tired i m so tired i m so tired i
Промпт: I want to
Дополнение LSTM: i want to go to the gym and i m so tired i m so tired i m so tired
Промпт: We have a flat in
Дополнение LSTM: we have a flat in the day i m so tired i m so tired i m so tired i


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Промпт: I cook dinner
Дополнение LSTM: i cook dinner and i m so tired i m so tired i m so tired i m so tired

Оценка автодополнения DistilGPT:
Промпт: My friend and I play
Дополнение DistilGPT: My friend and I play online as they go on and on, but the other two never really played
Промпт: I know who
Дополнение DistilGPT: I know who I am — and have to be. Let's put this place in perspective. I
Промпт: I want to
Дополнение DistilGPT: I want to share this post (no one can be sure about where this post began). If you
Промпт: We have a flat in
Дополнение DistilGPT: We have a flat in West Palm Beach. The flat is 6.5 feet long and covers about
Промпт: I cook dinner
Дополнение DistilGPT: I cook dinner with you soon but then, you'll not be ready to cook the dish...



* **LSTM** модель демонстрирует проблемы с логикой и связностью текста, часто повторяя одни и те же фразы, что снижает качество автодополнения.
* **DistilGPT** модель генерирует более содержательные и логичные продолжения, которые развивают идеи и создают интересный контекст.

# Оценка работы предобученного трансформера на тестовой выборке

In [4]:
# Устройство (CPU или GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transformer_rouge1, transformer_rouge2 = evaluate_transformer(test_loader, tokenizer, device=device, max_examples=1000)

DistilGPT-2: 100%|██████████| 1000/1000 [01:48<00:00,  9.25it/s]

DistilGPT-2 (на 1000 примерах):
  ROUGE-1: 0.6723
  ROUGE-2: 0.6203


# ВЫВОДЫ

В целом, **DistilGPT** показывает **значительно лучшие результаты** в плане качества и содержательности автодополнений по сравнению с LSTM.
<br> на это указывают как значения rouge-1 и rouge-2, которые в два раза превосходят значения для LSTM, так и непосредственное сравнение полученных текстов.
| Модель | rouge-1 | rouge-2|
|--------|---------|--------|
| LSTM| 0.3534 |  0.3050 |
| DistilGPT-2 | 0.6672 | 0.6164 |

На тестовой выборке DistilGPT-2 показала также хороший результат:
  * ROUGE-1: 0.6723;
  * ROUGE-2: 0.6203